# Train YOLOv1

In [1]:
import shutil
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import sys
sys.path.append('../')
from torchsummary.torchsummary import summary
from dataloader import VOC
import numpy as np
import matplotlib.pyplot as plt
import yolov1
from yolov1 import detection_collate
from yolov1 import detection_loss
from yolov1 import save_checkpoint

# 0. Vidom on

In [2]:
def create_vis_plot(_xlabel, _ylabel, _title, _legend):
    return viz.line(
        X=torch.zeros((1,)).cpu(),
        Y=torch.zeros((1, 1)).cpu(),
        opts=dict(
            xlabel=_xlabel,
            ylabel=_ylabel,
            title=_title,
            legend=_legend
        )
    )

In [3]:
def update_vis_plot(iteration, loss, window1, window2, update_type,
                    epoch_size=1):
    viz.line(
        X=torch.ones((1, 1)).cpu() * iteration,
        Y=torch.Tensor([loss]).unsqueeze(0).cpu() / epoch_size,
        win=window1,
        update=update_type
    )

In [4]:
import visdom
viz = visdom.Visdom()

vis_title = 'yolov1_martin_visioNoob.PyTorch on ' + 'VOC'
vis_legend = ['Train Loss']
iter_plot = create_vis_plot('Iteration', 'Loss', vis_title, vis_legend)
epoch_plot = create_vis_plot('Epoch', 'Loss', vis_title, vis_legend)

# 1. Device configuration

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

num_epochs = 16000
num_classes = 21
batch_size = 16
learning_rate = 1e-4

dropout_prop = 0.5

# 2. Load VOC Pascal'12 Dataset

In [6]:
DATASET_PATH_MARTIN = "/media/keti-ai/AI_HARD3/DataSets/VOC_Pascal/VOC/VOCdevkit/VOC2012"
DATASET_PATH_JAEWON = "D:/dataset/VOC2012"
train_dataset = VOC(root = DATASET_PATH_JAEWON,
                    transform=transforms.ToTensor(), cls_option = True, selective_cls="person")


VOC Parsing:   |████████████████████████████████████████| 100.0% (17125/17125)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (9583/9583)  Complete



In [7]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True,
                                           collate_fn=detection_collate)

# 3. Load YOLOv1

In [8]:
net = yolov1.YOLOv1()
# visualize_weights_distribution(net)

model = torch.nn.DataParallel(net, device_ids=[0]).cuda()

summary(model, (3, 448,448))

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           9,472
         LeakyReLU-2         [-1, 64, 224, 224]               0
         MaxPool2d-3         [-1, 64, 112, 112]               0
            Conv2d-4        [-1, 192, 112, 112]         110,784
         LeakyReLU-5        [-1, 192, 112, 112]               0
         MaxPool2d-6          [-1, 192, 56, 56]               0
            Conv2d-7          [-1, 128, 56, 56]          24,704
         LeakyReLU-8          [-1, 128, 56, 56]               0
            Conv2d-9          [-1, 256, 56, 56]         295,168
        LeakyReLU-10          [-1, 256, 56, 56]               0
           Conv2d-11          [-1, 256, 58, 58]          65,792
        LeakyReLU-12          [-1, 256, 58, 58]               0
           Conv2d-13          [-1, 512, 56, 56]       1,180,160
        LeakyReLU-14          [-1, 512,

# 4.Train the model

In [ ]:
total_step = len(train_loader)
for epoch in range(num_epochs):

    for i, (images, labels) in enumerate(train_loader):

        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)

        # Calc Loss
        loss = detection_loss(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #Visualize Every Iteration
        if(i != 0):
            update_vis_plot(i, loss.item(), iter_plot, epoch_plot, 'append')

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, learning rate: {}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          [param_group['lr'] for param_group in optimizer.param_groups]))
            update_vis_plot(i, loss.item(), epoch_plot, None, 'append')
    if(epoch == 100) or (epoch == 500) or (epoch == 1000) or (epoch == 2000) or (epoch == 4000) or (epoch == 8000) or (epoch == 14000):
        scheduler.step()
            

    if (epoch % 300) == 0:
        save_checkpoint({
            'epoch': epoch + 1,
            'arch': "YOLOv1",
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }, False, filename='checkpoint_{}.pth.tar'.format(epoch))
        
        
        

Epoch [1/16000], Step [100/599], Loss: 8.3873, learning rate: [0.0001]
Epoch [1/16000], Step [200/599], Loss: 7.3146, learning rate: [0.0001]
Epoch [1/16000], Step [300/599], Loss: 7.6288, learning rate: [0.0001]
Epoch [1/16000], Step [400/599], Loss: 9.8946, learning rate: [0.0001]
Epoch [1/16000], Step [500/599], Loss: 6.9923, learning rate: [0.0001]
Epoch [2/16000], Step [100/599], Loss: 7.6233, learning rate: [0.0001]
Epoch [2/16000], Step [200/599], Loss: 7.9447, learning rate: [0.0001]
Epoch [2/16000], Step [300/599], Loss: 10.4092, learning rate: [0.0001]
Epoch [2/16000], Step [400/599], Loss: 8.6986, learning rate: [0.0001]
Epoch [2/16000], Step [500/599], Loss: 9.1989, learning rate: [0.0001]
Epoch [3/16000], Step [100/599], Loss: 10.6552, learning rate: [0.0001]
Epoch [3/16000], Step [200/599], Loss: 9.7804, learning rate: [0.0001]
Epoch [3/16000], Step [300/599], Loss: 9.1337, learning rate: [0.0001]
Epoch [3/16000], Step [400/599], Loss: 7.7703, learning rate: [0.0001]
Epoc